Cell 1 – Imports & Settings

In [113]:
import os, time, random, re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tenacity import retry, stop_after_attempt, wait_exponential
from transformers import pipeline

# Basic scraper settings
BASE_HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/126.0 Safari/537.36"
    )
}

# Categories from Wadhefa
CATEGORIES = {
    "IT": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=75&jids%5B%5D=76&jids%5B%5D=84",
    "Marketing": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=72&jids%5B%5D=73&jids%5B%5D=74",
    "Finance": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=64&jids%5B%5D=77&jids%5B%5D=79",
    "Engineering": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=80&jids%5B%5D=81&jids%5B%5D=82",
    "Healthcare": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=96&jids%5B%5D=97",
    "Education": "https://www.wadhefa.com/jobfind.php?action=search&jids%5B%5D=67"
}

LINKS_PER_CATEGORY = 50  # limit for quick testing (increase later)


Cell 2 – Helper Functions

In [114]:
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=8))
def fetch(url: str) -> str:
    resp = requests.get(url, headers=BASE_HEADERS, timeout=25)
    resp.raise_for_status()
    return resp.text

def extract_links_from_list_page(list_url: str, limit: int = 50):
    """Extract job links from a Wadhefa listing page."""
    html = fetch(list_url)
    soup = BeautifulSoup(html, "lxml")
    links = []
    for a in soup.select('a.tablelist[href*="/details/job/"]'):
        href = a.get("href")
        if href:
            links.append(urljoin(list_url, href))
        if len(links) >= limit:
            break
    return list(dict.fromkeys(links))  # remove duplicates


Cell 3 – Text Normalization

In [115]:
AR_DIACRITICS_RE = re.compile(r"[\u064B-\u0652]")

def normalize_ar_keep_ascii(text: str) -> str:
    """Light cleanup for Arabic text (keep ASCII like emails/numbers)."""
    if not text:
        return ""
    t = text.replace("\u00A0"," ")
    t = re.sub(r"\u0640", "", t)     # tatweel
    t = AR_DIACRITICS_RE.sub("", t)  # diacritics
    t = t.replace("أ","ا").replace("إ","ا").replace("آ","ا") \
         .replace("ى","ي").replace("ؤ","و").replace("ئ","ي")
    t = re.sub(r"[ \t]+", " ", t).strip()
    return t


Cell 4 – Job Detail Parser

In [116]:
def parse_job_detail(job_url: str, category_label: str):
    html = fetch(job_url)
    soup = BeautifulSoup(html, "lxml")

    # description
    d_el = soup.select_one("td.td4textarea")
    desc_raw = d_el.get_text("\n", strip=True) if d_el else ""
    description = normalize_ar_keep_ascii(desc_raw)

    return {
        "category": category_label,
        "description": excel_safe(description),
        "url": job_url,
    }


Cell 5 – Track Seen Jobs

In [117]:
def load_seen_jobs(path="seen_jobs.txt"):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return set(f.read().splitlines())
    except FileNotFoundError:
        return set()

def save_seen_jobs(seen_jobs, path="seen_jobs.txt"):
    with open(path, "w", encoding="utf-8") as f:
        for url in seen_jobs:
            f.write(url + "\n")


In [118]:
import os
print(os.getcwd())

c:\Users\maanx\Documents\VS code\samsung project\job_posts_part


Cell 6 – Load Job Classifier

In [119]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_path = "./job_classifier"   # since you're already inside job_posts_part

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

job_classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    function_to_apply="softmax",
    truncation=True,   # 🔹 cut text if too long
    max_length=512     # 🔹 AraBERT max length
)


# quick test
test_text = "مطلوب مطور بايثون للعمل على تطبيقات ويب"
print(job_classifier(test_text))


Device set to use cpu


[{'label': 'LABEL_0', 'score': 0.9578908085823059}]


In [120]:
import pandas as pd

# Load the training dataset categories (used when training the job classifier)
df_jobs = pd.read_csv("wadhefa_dataset.csv")  

# Build label mappings
label_list = df_jobs['category'].unique().tolist()
label_to_id = {label: idx for idx, label in enumerate(label_list)}
id_to_label = {idx: label for label, idx in label_to_id.items()}

print("Job classifier label mapping:", id_to_label)


Job classifier label mapping: {0: 'IT', 1: 'Marketing', 2: 'Finance', 3: 'Engineering', 4: 'Healthcare', 5: 'Education'}


Cell 7 – Classification Helper

In [121]:
def classify_job(description: str) -> str:
    if not description or not description.strip():
        return "Unknown"

    result = job_classifier(description)[0]  # e.g., {'label': 'LABEL_0', 'score': 0.97}

    if result["label"].startswith("LABEL_"):
        label_idx = int(result["label"].split("_")[-1])
        return id_to_label.get(label_idx, "Unknown")

    if result["label"] in id_to_label.values():
        return result["label"]

    return "Unknown"


Cell 8 – Main Scraper + Classifier Loop

In [122]:
all_rows = []
seen = load_seen_jobs()

for label, list_url in CATEGORIES.items():
    print(f"Scraping category: {label}")
    links = extract_links_from_list_page(list_url, limit=LINKS_PER_CATEGORY)
    new_links = [u for u in links if u not in seen]

    print(f"  Found {len(new_links)} new jobs")
    for u in new_links:
        try:
            row = parse_job_detail(u, category_label=label)
            row["predicted_category"] = classify_job(row["description"])  # 🔹 classify immediately
            all_rows.append(row)
            seen.add(u)
            time.sleep(random.uniform(0.6, 1.2))
        except Exception as e:
            print("  Failed:", u, e)

save_seen_jobs(seen)


Scraping category: IT
  Found 0 new jobs
Scraping category: Marketing
  Found 0 new jobs
Scraping category: Finance
  Found 0 new jobs
Scraping category: Engineering
  Found 0 new jobs
Scraping category: Healthcare
  Found 0 new jobs
Scraping category: Education
  Found 0 new jobs


Cell 9 – Save to CSV

In [123]:
if all_rows:
    new_df = pd.DataFrame(all_rows).drop_duplicates()

    # Save new jobs only
    new_df.to_csv("new_classified_jobs.csv", index=False, encoding="utf-8-sig")

    # Append/merge into full dataset
    if os.path.exists("classified_wadhefa_dataset.csv"):
        full_df = pd.concat([pd.read_csv("classified_wadhefa_dataset.csv"), new_df], ignore_index=True).drop_duplicates()
    else:
        full_df = new_df

    full_df.to_csv("classified_wadhefa_dataset.csv", index=False, encoding="utf-8-sig")
    print("✅ Saved updated dataset with classified jobs")
else:
    print("No new jobs found.")


No new jobs found.


In [124]:
import pandas as pd

# Load the classified jobs dataset
df = pd.read_csv("classified_wadhefa_dataset.csv")

# Take the first job description
sample_text = df["description"].iloc[0]
print("Sample job description:\n", sample_text, "\n")

# Run it through the classifier
result = job_classifier(sample_text)
print("Raw classifier output:", result)


Sample job description:
 - تحديد مواصفات الحلول البديلة والاضافات والتحديثات اللازمة ويدرس جدواها فنيا واقتصاديا
- مراقبة الاستخدام والابلاغ عن المشكلات والاخطاء والعمل علي حلها والتاكد من الاستخدام السليم
- تنفيذ احدث التطبيقات والنظم في مجال تقنية وامن المعلومات وجدولة دورية لصيانة نظم المعلومات ووضع برامج الدعم والصيانة
- تنسيق التعاقدات التقنية وعمليات الشراء الخاصة بها
- القيام بعمليات تحديد احتياجات المستخدمين في مختلف الوحدات التنظيمية في الجهة والقيام بمراجعتها وتقديم الاقتراحات المعنية بتطوير ووضع المواصفات التقنية للانظمة والتطبيقات اللازمة والقيام بعملها
- اعداد برامج وخطط العمل المتعلقة باعمال تقنية المعلومات وعمليات تصميم وتطوير البرامج والتطبيقات والقيام بها وعمل انشطة دعم المستخدم بهدف توفير الحلول التقنية اللازمة عند مواجهة اية صعوبات او مشاكل
- متابعة اخر المستجدات والتطورات المتعلقة بشوون وعمليات تقنية المعلومات والتوجيه بتبني المناسب منها، وتحديد وضمان توافر جميع الموارد والمواد اللازمة لتنفيذ الاعمال والانشطة التخصصية باحدث الاساليب والممارسات ذات العلاقة
- القيام ب

In [127]:
import pandas as pd

# Load the dataset
df = pd.read_csv("classified_wadhefa_dataset.csv")

# Drop the title column if it exists
if "title" in df.columns:
    df = df.drop(columns=["title"])

# Save cleaned dataset
df.to_csv("classified_wadhefa_dataset.csv", index=False, encoding="utf-8-sig")

print("Cleaned dataset saved. New columns:", df.columns.tolist())


Cleaned dataset saved. New columns: ['category', 'description', 'url', 'predicted_category']


In [128]:
import pandas as pd

# Load the file
df = pd.read_csv("classified_wadhefa_dataset.csv")

# Keep only rows where predicted_category != "Unknown"
df_clean = df[df["predicted_category"] != "Unknown"]

# Save back
df_clean.to_csv("classified_wadhefa_dataset.csv", index=False, encoding="utf-8-sig")

print("Cleaned dataset saved. Remaining rows:", df_clean.shape[0])
print(df_clean["predicted_category"].value_counts())


Cleaned dataset saved. Remaining rows: 3
predicted_category
IT           2
Marketing    1
Name: count, dtype: int64
